# <h1 align="center"><font color="red">Agentes de IA vs. Pipelines de IA - Um guia prático para programar sua aplicação de LLM</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Usamos o ``CrewAI`` para criar aplicativos que demonstram como escolher a arquitetura certa para sua aplicação ``LLM``.

Podemos pensar em um ``agente de IA`` como um ``LLM`` com acesso a ferramentas externas; ele funciona em um loop, tomando decisões sobre como se comportar e quais ferramentas usar em cada iteração.

Ao fazer isso, o agente pode resolver problemas muito mais complexos do que um aplicativo ``LLM`` convencional. (Expliquei como construir um agente desse tipo do zero no artigo [Como construir um agente ReAct AI com Claude 3.5 e Python](https://ai.gopubby.com/how-to-build-a-react-ai-agent-with-claude-3-5-and-python-95423f798640))

``Agentes de IA`` são poderosos e estão muito além de um aplicativo de bate-papo convencional. Mas nem sempre são a solução certa.

<font color="orange">Às vezes, uma sequência de funções mais convencionais é mais apropriada. Podemos pensar nesse tipo de aplicação como um ``pipeline``, onde uma solução é desenvolvida passando a saída de uma função como entrada para a próxima.</font>

Neste artigo, exploraremos o uso de ``agentes`` e ``pipelines`` de IA e para quais tipos de aplicações eles são mais adequados. Usaremos a estrutura de código aberto ``CrewAI`` para criar a lógica do ``LLM`` e construir um front-end simples em ``Streamlit`` para um aplicativo online.

# <font color="gree">Aplicativos para agentes e aplicativos para pipelines</font>

Vamos pensar em alguns aplicativos diferentes. 

Um cliente acessa o site de um fabricante para resolver um problema com sua máquina de lavar. Ele é recebido por um ``agente de IA`` que quer ajudar. Qual é a marca e o modelo da máquina? Quais são os sintomas? As roupas ainda estão sujas? Há água no chão? Ela não centrifuga?

Há uma série de perguntas que o agente pode fazer para resolver qualquer problema do cliente. ``E as perguntas subsequentes dependerão das respostas anteriores.``

Nesse caso, um ``agente de IA`` é uma ótima solução. Ele precisa trabalhar em loop, coletando informações, até que esteja em condições de sugerir uma solução ou, caso fique sem ideias, repassar o problema a um operador humano.

O funcionamento deste tipo de agente está representado no diagrama abaixo.

![](https://datavizandai.github.io/assets/images/agentpipeline/reactflow.png)

``Aqui está outro cenário.``

A diretora de vendas de uma pequena empresa que administra cafeterias em Londres quer produzir um relatório sobre o desempenho de várias filiais. Ela precisa verificar regularmente como o desempenho difere entre as lojas localizadas no distrito financeiro da City e o West End, mais voltado para o turismo e o varejo. Ela tem planilhas detalhando as vendas de cada filial. Ela quer consolidar os números e escrever um relatório para sua diretoria, que inclua gráficos de fácil compreensão que mostrem o desempenho relativo de cada cafeteria.

Nesse caso, o diretor poderia usar um aplicativo LLM para combinar os números de vendas mensais em uma única entidade, escrever um comentário sobre o desempenho dos diferentes pontos de venda, produzir os gráficos apropriados e combinar esses elementos em um único relatório. O processo é o mesmo todos os meses e a única variável são os valores nas planilhas. Portanto, embora o conteúdo do relatório varie, o processo é o mesmo.

``Diferentemente do exemplo agêntico, este é um fluxo sequencial simples.``


![](https://datavizandai.github.io/assets/images/agentpipeline/pipelineflow.png)

Um ``agente de IA`` é a solução ideal para o primeiro problema. Dispondo de informações adequadas sobre o funcionamento da máquina de lavar, seus potenciais modos de falha e os sintomas dessas falhas, o agente pode buscar informações de forma inteligente e formular novas perguntas em resposta às informações do cliente até encontrar uma solução. Para esse tipo de problema, o ``agente de IA`` oferece uma solução flexível e inteligente.

Um relatório de vendas é um assunto diferente. Os números de vendas mudam e pode ser que um relatório encontre diferenças no desempenho dependendo da época do ano e da frequência de turistas ou trabalhadores de escritório.

Portanto, embora seja necessária uma visão inteligente desses números variáveis, os processos para a criação do relatório são os mesmos. Consolidar os números; analisar os resultados de vendas; criar os gráficos; e, por fim, redigir o relatório completo.

Você pode escrever um prompt para um agente que o oriente a seguir as etapas necessárias, mas um resultado mais consistente pode vir da criação de uma série de tarefas em que a saída de uma alimenta a entrada da próxima.

# <font color="yellow">CrewAI</font>

O ``CrewA``I está disponível em duas versões:`` Enterprise`` e ``Open Source``. Usaremos a versão de código aberto aqui, portanto, não há chaves de API ou custos envolvidos.

A instalação do ``CrewAI`` é o que você esperaria, embora, como usaremos ferramentas externas, precisamos adicionar essa funcionalidade também.

```
pip install crewai crewai-tools
```
Criaremos uma ferramenta para o ``CrewAI`` usar com base na ``biblioteca Wikipedia do PyPi``, então também precisaremos instalá-la.

```
pip install wikipedia
```

O primeiro aplicativo que desenvolveremos se chama ``Wikipédia Inteligente``. Ele realizará uma consulta e, em seguida, usará a ferramenta ``Wikipédia`` para encontrar uma resposta. Ele fará isso raciocinando sobre a consulta, calculando o que precisa descobrir, realizando as buscas apropriadas e analisando os resultados antes de fornecer uma resposta. Se isso parece complicado, não se preocupe, não é.

## <font color="pink">Wikipédia inteligente</font>


In [1]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file


### <font color="green">Tools</font>

Agora vamos especificar a ``ferramenta Wikipédia``. Uma ferramenta é uma função que um agente CrewAI pode chamar, se necessário. Ela executa uma tarefa e retorna um valor. A maneira mais simples de definir uma ferramenta é com um ``decorador``. Então, a próxima célula é:

In [2]:
from crewai.tools import tool
import wikipedia

@tool("wikipedia_lookup")
def wikipedia_lookup(q: str) -> str:
    """Olhe para uma consulta no Wikipedia e retorne o resultado"""
    return wikipedia.page(q).summary

Após as importações, temos uma função simples que recebe um parâmetro de string, o envia para a ``Wikipédia`` e retorna um resumo de uma página da ``Wikipédia``. Observe que incluímos uma descrição da função e dicas de tipo para o parâmetro e o valor de retorno. Isso ajuda o ``CrewAI`` a entender como usar a ferramenta.

Agora chegamos à parte principal da aplicação: definimos o ``agente`` que fará o trabalho para nós e a tarefa que queremos que ele execute, e então criamos e executamos a equipe. ``Essa equipe combina agentes e tarefas em algo que pode ser executado.`` Em nossa primeira aplicação, trata-se de um único agente e uma única tarefa, mas mais adiante veremos equipes mais complexas .

### <font color="green">Agentes</font>

Vamos primeiro analisar o agente. O código a seguir cria uma variável de agente chamada ``researcher_agent``. Ela especifica uma ``função``, um ``objetivo``, uma ``história de fundo`` e uma ``lista de ferramentas`` que o agente pode usar. Ela também especifica o modelo de LLM a ser usado. Portanto, a próxima célula é:

In [3]:
researcher_agent = Agent(
        role="Pesquisador",
        goal="Você pesquisa tópicos usando Wikipedia e relata os resultados",
        backstory="Você é um escritor e editor experiente",
        tools=[wikipedia_lookup],
        llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0) # "gpt-4o"   ou  "gpt-4o-mini"  ou  "gpt-3.5-turbo-0125"
    )


Muitos outros atributos podem ser especificados para um Agente, mas estes são os básicos. O ``LLM`` e as ferramentas são bastante óbvios, sendo itens sobre os quais você deve ser específico. Mas é compreensível pensar que ``role`` , ``goal`` e ``backstory`` são mais como encantamentos mágicos do que requisitos de programação. No entanto, como você pode esperar, essas strings são usadas quando o ``CrewAI`` constrói ``prompts`` para o ``LLM``.

Então, isso é um Agente: algumas coisas que informam ao ``LLM`` sua finalidade, uma lista de ferramentas e um modelo de ``LLM``.

### <font color="green">Tasks</font>

A próxima etapa é a ``task``. Ela nos diz o que o agente precisa fazer e, para nossos propósitos, envolverá responder a uma consulta. Quando criarmos um aplicativo adequado, solicitaremos uma consulta ao usuário, mas este é um protótipo do ``Jupyter Notebook``, então, para facilitar, codificaremos a consulta.

In [4]:
query = "Nestes dias que ex-presidente foi preso no Perú?"

Essa não é uma pergunta que pode ser respondida diretamente em uma página da Wikipédia. Será necessário algum ``raciocínio``.

E aqui está a especificação da tarefa que incorporará esta consulta.

In [5]:
task1 = Task(
    description=query,
    expected_output='Um texto curto baseado nas informações da ferramenta.',
    agent=researcher_agent,
    tools=[wikipedia_lookup]
)

A descrição da tarefa é a consulta, explicamos qual será a saída esperada (``essa é outra sequência mágica que terminará em um prompt``), associamos a tarefa a um agente e listamos as ferramentas que podem ser usadas.

Agora amarramos tudo isso com uma crew (equipe).

### <font color="green">Crews</font>

In [6]:
crew = Crew(
    agents=[researcher_agent],
    tasks=[task1],
    verbose=False
)

Inclui uma lista de ``agentes`` e uma lista de ``tarefas``. Neste caso, uma tarefa e um agente. Quando o ``verbose`` sinalizador for definido, o ``CrewAI`` mostrará seu funcionamento — veremos isso em breve.

Agora tudo o que precisamos fazer é executar tudo e analisar os resultados.

In [8]:
result = crew.kickoff()
print(result)
print("\n\n")
# Acessando a saída da tarefa:
task_output = task1.output

print(f"Descrição da tarefa: {task_output.description}")
print(f"Resumo da tarefa: {task_output.summary}")
print(f"Raw Output: {task_output.raw}")

O ex-presidente do Peru, José Pedro Castillo, foi preso em 7 de dezembro de 2022, após tentar dissolver ilegalmente o Congresso e governar por decreto. Ele foi destituído do cargo pelo Congresso e detido sob acusações de sedição e alta traição.



Descrição da tarefa: Nestes dias que ex-presidente foi preso no Perú?
Resumo da tarefa: Nestes dias que ex-presidente foi preso no Perú?...
Raw Output: O ex-presidente do Peru, José Pedro Castillo, foi preso em 7 de dezembro de 2022, após tentar dissolver ilegalmente o Congresso e governar por decreto. Ele foi destituído do cargo pelo Congresso e detido sob acusações de sedição e alta traição.


<font color="orange">Você pode ver que o `CrewAI` segue um processo muito semelhante ao de um agente. Ele raciocina sobre sua resposta, coleta as informações apropriadas e, em seguida, responde com uma resposta adequada.</font>